In [8]:
%pip install torch
%pip install numpy
%pip install pandas
%pip install scikit-learn
%pip install coremltools

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from numpy import genfromtxt
from sklearn.preprocessing import LabelEncoder

In [10]:
# Set up the paths
HOME_PATH = os.path.expanduser('~')
MODELS_PATH = f'{HOME_PATH}/Developer/BU/research/models'
DATASET_PATH = f'../../../data/'
data_features = f'{DATASET_PATH}/WISDM_x.csv'
data_labels = f'{DATASET_PATH}/WISDM_y.csv'

In [11]:
# Load Data
x = genfromtxt(data_features, delimiter=',')
y_df = pd.read_csv(data_labels)
y = y_df.values.flatten()  # Flatten if y is 2D

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Function to create time series dataset
def create_series(x, y, timestep, overlap):
    slide_step = int(timestep * (1 - overlap))
    data_num = int((len(x) / slide_step) - 1)
    dataset = np.ndarray(shape=(data_num, timestep, x.shape[1]))
    labels = []

    for i in range(data_num):
        labels.append(y[slide_step * (i + 1) - 1])
        for j in range(timestep):
            dataset[i, j, :] = x[slide_step * i + j, :]

    return dataset, np.array(labels)

# Create time series
timestep = 16  # Replace with your value
overlap = 0.5  # Replace with your value
X_series, y_series = create_series(x, y_encoded, timestep, overlap)

In [12]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_series, y_series, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
x_train_tensor = torch.tensor(X_train, dtype=torch.float32)
x_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [13]:
print(f'X_train.shape:{X_train.shape}, X_test.shape:{X_test.shape}, y_train.shape:{y_train.shape}, y_test.shape:{y_test.shape}')

X_train.shape:(104856, 16, 3), X_test.shape:(26214, 16, 3), y_train.shape:(104856,), y_test.shape:(26214,)


In [14]:
# Define the MLP model
class MyMLP(nn.Module):
    def __init__(self, input_size, num_classes=6):
        super(MyMLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Model Initialization
input_size = timestep * X_series.shape[2]  # Calculate input size
model = MyMLP(input_size)

# DataLoader
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)



In [15]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Training function
def train(model, train_loader, criterion, optimizer, epochs=100):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch}, Loss: {total_loss / len(train_loader)}')

# Train the model
train(model, train_loader, criterion, optimizer)


Epoch 0, Loss: 0.5094858575532755
Epoch 1, Loss: 0.3675977605746751
Epoch 2, Loss: 0.3169934956255739
Epoch 3, Loss: 0.28529085023436623
Epoch 4, Loss: 0.26287043613084504
Epoch 5, Loss: 0.24820865482282828
Epoch 6, Loss: 0.23512211444721198
Epoch 7, Loss: 0.22353049599275202
Epoch 8, Loss: 0.21587345948473502
Epoch 9, Loss: 0.20771914165037444
Epoch 10, Loss: 0.1991336875350247
Epoch 11, Loss: 0.19441769759626998
Epoch 12, Loss: 0.1875261401029132
Epoch 13, Loss: 0.1831271066782113
Epoch 14, Loss: 0.17878861868636975
Epoch 15, Loss: 0.17520304096156936
Epoch 16, Loss: 0.1710243168570524
Epoch 17, Loss: 0.16520355892429597
Epoch 18, Loss: 0.1639568588876548
Epoch 19, Loss: 0.15894396949782194
Epoch 20, Loss: 0.15904942914602632
Epoch 21, Loss: 0.1547117856540845
Epoch 22, Loss: 0.1516001032627869
Epoch 23, Loss: 0.15000481451074568
Epoch 24, Loss: 0.14704957723963036
Epoch 25, Loss: 0.14400741521921406
Epoch 26, Loss: 0.14195667339865028
Epoch 27, Loss: 0.13847401537980755
Epoch 28, Lo

In [16]:
model_path =  f'{MODELS_PATH}/MLP_base.pth'
torch.save(model.state_dict(), model_path)

In [17]:
def evaluate(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            loss = criterion(output, target)
            total_loss += loss.item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'Test set: Average loss: {total_loss / len(test_loader)}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)')

# DataLoader for test set
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Evaluate the model
evaluate(model, test_loader, criterion)


Test set: Average loss: 0.4916046664663931, Accuracy: 23973/26214 (91%)


#### Coreml evaluation

In [22]:
from pathlib import Path

def coreml_metrics(model_name, X_test, y_test, model_path):
    predictions = []
    for id in range(len(X_test)):
        X_test_new = np.expand_dims(X_test[id], axis=0)
        output_dict = model_name.predict({'x': X_test_new})
        pred_class = np.argmax(output_dict['linear_2'])
        predictions.append(pred_class)
    
    accuracy = np.sum(predictions == y_test) / len(predictions)
    print("Accuracy:", accuracy)
    
    model_file = Path(model_path)
    
    # Size in bytes
    model_size_bytes = model_file.stat().st_size
    
    # Convert size to kilobytes (optional)
    model_size_kb = model_size_bytes / 1024
    print(f"Size of the model: {model_size_kb:.2f} KB")

### Convert to Coreml

In [37]:
import coremltools as ct
example_input = torch.rand(1,16, 3) 

model.eval()
traced_model = torch.jit.trace(model, example_input)
out = traced_model(example_input)

# Convert to Core ML program using the Unified Conversion API.
mlp_coreml_model = ct.convert(
    traced_model,
    convert_to="mlprogram",
    inputs=[ct.TensorType(shape=example_input.shape)]
 )

# Save the converted model.
mlp_coreml_model.save(f'{MODELS_PATH}/mlp.mlpackage')

Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 16572.82 passes/s]


In [38]:
import coremltools as ct
import coremltools.optimize.coreml as cto

mlp_coreml_model = ct.models.MLModel(f'{MODELS_PATH}/mlp.mlpackage')

In [23]:
model_name = mlp_coreml_model
model_path = f'{MODELS_PATH}/mlp.mlpackage'

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.9145494773785
Size of the model: 0.12 KB


In [24]:
import os
os.path.getsize(f'{MODELS_PATH}/mlp.mlpackage')


128

## Post Training Optimization

In [39]:
from coremltools.optimize.coreml import (
    OpThresholdPrunerConfig,
    OpMagnitudePrunerConfig,
    OpPalettizerConfig,
    OpLinearQuantizerConfig,
    OptimizationConfig,
    prune_weights,
)

### Quantization

In [40]:
op_config = OpLinearQuantizerConfig(
    mode="linear_symmetric", weight_threshold=512
)
config = OptimizationConfig(global_config=op_config)

compressed_8_bit_model = cto.linear_quantize_weights(mlp_coreml_model, config=config)

Running compression pass linear_quantize_weights: 100%|██████████| 5/5 [00:00<00:00, 1750.69 ops/s]
Running MIL frontend_milinternal pipeline: 0 passes [00:00, ? passes/s]
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 8517.79 passes/s]


In [41]:
compressed_8_bit_model.save(f'{MODELS_PATH}/mlp_8bitQuantized_mlmodel.mlpackage')

In [42]:
model_name = compressed_8_bit_model
model_path = f'{MODELS_PATH}/mlp_8bitQuantized_mlmodel.mlpackage'

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.9143205920500496
Size of the model: 0.12 KB


### Pruning

#### a) OpMagnitudePrunerConfig: Prune the weights with a constant sparsity percentile

In [43]:
op_config = OpMagnitudePrunerConfig(
    target_sparsity=0.6,
    weight_threshold=1024,
)
config = OptimizationConfig(global_config=op_config)
mlp_magnitude_pruner = prune_weights(mlp_coreml_model, config=config)


Running compression pass prune_weights: 100%|██████████| 5/5 [00:00<00:00, 1398.94 ops/s]
Running MIL frontend_milinternal pipeline: 0 passes [00:00, ? passes/s]
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 12110.60 passes/s]


In [44]:
mlp_magnitude_pruner.save(f'{MODELS_PATH}/mlp_magnitude_pruner.mlpackage')

In [45]:
model_name = mlp_magnitude_pruner
model_path = f'{MODELS_PATH}/mlp_magnitude_pruner.mlpackage'

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.6240558480201419
Size of the model: 0.12 KB


#### b) OpThresholdPrunerConfig: Sets all weight values below a certain value.

In [46]:
op_config = OpThresholdPrunerConfig(
    threshold=0.001,
    minimum_sparsity_percentile=0.01,
    weight_threshold=1024,
)

config = OptimizationConfig(global_config=op_config)
mlp_threshold_pruner = prune_weights(mlp_coreml_model, config=config)

Running compression pass prune_weights:   0%|          | 0/5 [00:00<?, ? ops/s]weight value has sparsity of 0.0016276041666666667 < minimum_sparsity_percentile 0.01. Skipped.
weight value has sparsity of 0.0025634765625 < minimum_sparsity_percentile 0.01. Skipped.
Running compression pass prune_weights: 100%|██████████| 5/5 [00:00<00:00, 3048.63 ops/s]
Running MIL frontend_milinternal pipeline: 0 passes [00:00, ? passes/s]
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 18098.40 passes/s]


In [47]:
mlp_threshold_pruner.save(f'{MODELS_PATH}/mlp_threshold_pruner.mlpackage')

In [48]:
model_name = mlp_threshold_pruner
model_path = f'{MODELS_PATH}/mlp_threshold_pruner.mlpackage'

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.9145494773785
Size of the model: 0.12 KB


### Palletization 


In [49]:
op_config = OpPalettizerConfig(
    mode="kmeans",
    nbits=6
)

config = OptimizationConfig(global_config=op_config)
mlp_palettizer = cto.palettize_weights(mlp_coreml_model, config=config)

Running compression pass palettize_weights: 100%|██████████| 5/5 [00:00<00:00, 119.10 ops/s]
Running MIL frontend_milinternal pipeline: 0 passes [00:00, ? passes/s]
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 7793.69 passes/s]


In [51]:
mlp_palettizer.save(f'{MODELS_PATH}/mlp_palettizer.mlpackage')


In [52]:
model_name = mlp_palettizer
model_path = f'{MODELS_PATH}/mlp_palettizer.mlpackage'

coreml_metrics(model_name, X_test, y_test, model_path)


Accuracy: 0.912489509422446
Size of the model: 0.12 KB


In [25]:
# define op config 
op_config = cto.OpPalettizerConfig(mode="kmeans", nbits=6)

# define optimization config by applying the op config globally to all ops 
config = cto.OptimizationConfig(global_config=op_config)

# palettize weights
#compressed_mlmodel = cto.palettize_weights(mlmodel, config)
compressed_mlmodel = cto.palettize_weights(mlp_coreml_model, config)

compressed_mlmodel.save(f'{MODELS_PATH}/mlp_palletization.mlpackage')

Running compression pass palettize_weights: 100%|██████████| 5/5 [00:03<00:00,  1.41 ops/s]
Running MIL frontend_milinternal pipeline: 0 passes [00:00, ? passes/s]
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 9780.73 passes/s]


In [26]:
import coremltools as ct
import coremltools.optimize.coreml as cto

compressed_mlmodel = ct.models.MLModel(f'{MODELS_PATH}/mlp_palletization.mlpackage')

In [53]:
model_name = compressed_mlmodel
model_path = f'{MODELS_PATH}/mlp_palletization.mlpackage'

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.912489509422446
Size of the model: 0.12 KB
